# Solving Sudoku Puzzles
## Overview
Sudoku is a popular logic-based number puzzle. It consists of a 9x9 grid, divided into nine 3x3 subgrids. Some cells contain numbers, while others are empty. The objective is to fill the empty cells with numbers from 1 to 9, ensuring each number appears exactly once in each row, column, and 3x3 subgrid.

## Writing an Efficient Sudoku Solver
To tackle the challenge of solving Sudoku puzzles programmatically, I developed an efficient Sudoku solver. The approach and algorithm used can greatly influence the solver's performance and accuracy.

## Implementation Steps

1. **Grid Representation**: Use a 9x9 matrix to represent the Sudoku grid, where zeros represent empty cells.
2. **Constraint Propagation**: Implement constraint propagation to reduce the search space by eliminating impossible values for each cell.
3. **Backtracking Search**: Develop a recursive backtracking function that attempts to fill the grid, backtracking when a constraint violation is detected.
4. **Optimisation**: Enhance the solver with techniques such as forward-checking and the Minimum Remaining Value heuristic to improve efficiency.
5.**Testing and Validation**: Test the solver with a variety of Sudoku puzzles of varying difficulty to ensure robustness and performance.

## Sample Puzzles

To validate the solver, I used a set of sample Sudoku puzzles. Each puzzle is represented as a 9x9 NumPy array, with zeros indicating empty cells. The sample set includes puzzles of different difficulty levels, providing a comprehensive test bed for the solver.

## Performance Considerations

To achieve a high grade, the solver's speed is a critical factor. Efficient implementation of constraint satisfaction and optimization techniques is essential for handling more challenging puzzles within a reasonable timeframe.


In [ ]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

## Part One
All of the code for my Sudoku Solver are in the cell below.

I have included a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [ ]:
import numpy as np

#function verifies that each row is valid 
def checkrows(grid):
    for row in grid:
        if set(row) != set(range(1, 10)): #checks to ensure that there are no duplicates or invalid values
            return False                  #this is done by comparing the row to the full range o values 1 to 9
    return True

#function verifies that each column is valid
def checkcolumns(grid):
    for index in range(9):
        column = [row[index] for row in grid] 
        if set(column) != set(range(1, 10)): #same method of checking as for rows
            return False
    return True

#function verifies that each 3x3 square is valid
def checksquares(grid):
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            square = [grid[x][y] for x in range(i, i + 3) for y in range(j, j + 3)]
            if set(square) != set(range(1, 10)): #same method of checking as for rows & columns
                return False
    return True

#function that carries out hidden single and loops through cells in the board
def hiddensingles(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0: #checks for empty cell
                domain = constraintpropogatedvalues(board, i, j)
                for value in domain:
                    if findhiddensingles(board, i, j, value):
                        board[i][j] = int(value)
                        break

#checking through domains in rows, columns and 3x3s to find unique values
def findhiddensingles(board, row, col, value):
    valuesinrow = all(value not in constraintpropogatedvalues(board, row, x) for x in range(9) if x != col and board[row][x] == 0) #checking through domains of empty cells in row
    valuesincolumn = all(value not in constraintpropogatedvalues(board, x, col) for x in range(9) if x != row and board[x][col] == 0) #checking through domains of empty cells in column
    boxstartrow, boxstartcol = 3 * (row // 3), 3 * (col // 3)
    valueinbox = all(value not in constraintpropogatedvalues(board, r, c) for r in range(boxstartrow, boxstartrow + 3) for c in range(boxstartcol, boxstartcol + 3) if (r, c) != (row, col) and board[r][c] == 0)
    
    return valuesinrow or valuesincolumn or valueinbox

#constraint propogation 
def constraintpropogatedvalues(board, row, column):
    domain = set(map(str, range(1, 10))) #contraint satisfaction, starts establishes same domain of values 1 - 9
    domain -= set(map(str, board[row])) #removes other values in that row
    domain -= set(map(str, [board[x][column] for x in range(9)])) #removes other values in that column
    boxstartrow, boxstartcol = 3 * (row // 3), 3 * (column // 3) 
    #removes other values in the same 3x3 area
    domain -= set(map(str, [board[x][y] for x in range(boxstartrow, boxstartrow + 3) for y in range(boxstartcol, boxstartcol + 3)]))
    return domain

#MRV function
def minimumremainingcell(board):
    emptycells = [(i, j) for i in range(9) for j in range(9) if board[i][j] == 0] #getting positions of empty cells
    if not emptycells:
        return None #case for when sudoku is solved

    #sorts cells based on the domain of values for each
    sortedcells = sorted(emptycells, key=lambda cell: len(constraintpropogatedvalues(board, cell[0], cell[1]))) 
    return sortedcells #returns list of cells based on size of domain

#validation function to ensure that domain choice is suitable
def validate(board, row, column, val):
    #performs checks in both the row,column and cell
    rowcheck = val not in board[row] 
    columncheck = val not in [board[i][column] for i in range(9)]
    boxstartrow, boxstartcol = 3 * (row // 3), 3 * (column // 3) 
    boxcheck = val not in [board[i][j] for i in range(boxstartrow, boxstartrow+3) for j in range(boxstartcol, boxstartcol+3)]
    return rowcheck and columncheck and boxcheck

#main solver function 
def solve(board, row=0, column=0):
    remainingcells = minimumremainingcell(board) #processes cells in order of MRV
    if remainingcells is None:
        return True #solved is True

    row, column = remainingcells[0]

    possibilities = constraintpropogatedvalues(board, row, column)  #establishing domain of values for that cell
    for val in possibilities: 
        if validate(board, row, column, int(val)): #checks validity 
            board[row][column] = int(val) #places value in 
            if solve(board):
                return True
            board[row][column] = 0 #else backtrack

    return False #else unsolveable

def sudoku_solver(sudoku):
    if isinstance(sudoku, np.ndarray):
        sudoku = sudoku.tolist() #converts to list of lists 

    while True: 
        oldsudoku = [row.copy() for row in sudoku] #creates copy
        
        hiddensingles(sudoku)

        if sudoku == oldsudoku:
            break #if sudoku is unchanged then break loop

    #checking to see if sudoku is valid as well as whether sudoku was solved
    if not solve(sudoku) or not checkrows(sudoku) or not checkcolumns(sudoku) or not checksquares(sudoku):
        return np.full((9, 9), -1.0) #return 9x9 or -1s if unsolveable
    else:
        return np.array(sudoku) #otherwise returns solved sudoku


### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, my function should return a 9x9 NumPy array whose values are all equal to -1.

When I will test my code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether my code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate my code because it is taking too long.

The best way to improve the performance of my code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test the ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run the solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [ ]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    import numpy as np
    __SCORES = {}
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            if not isinstance(your_solution, np.ndarray):
                print("\033[91m[ERROR] Your sudoku_solver function returned a variable that has the incorrect type. If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns a {} object when {} was expected.\n\x1b[m".format(type(your_solution), np.ndarray))
            elif not np.all(your_solution.shape == (9, 9)):
                print("\033[91m[ERROR] Your sudoku_solver function returned an array that has the incorrect shape.  If you submit this it will likely fail the auto-marking procedure result in a mark of 0 as it is expecting the function to return a numpy array with a shape (9,9).\n\t\033[94mYour function returns an array with shape {} when {} was expected.\n\x1b[m".format(your_solution.shape, (9, 9)))
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took {} seconds to solve.\n".format(end_time-start_time))

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        __SCORES[difficulty] = {
            'correct': count,
            'total': len(sudokus)
        }

## Submission Test

In [ ]:
import sys
import pathlib

fail = False;

success = '\033[1;32m[✓]\033[0m'
issue = '\033[1;33m[!]'
error = '\033[1;31m\t✗'
indent_success = '\033[1;32m\t✓'

#######
##
## Skip Tests check.
##
## Test to ensure the SKIP_TESTS variable is set to True to prevent it slowing down the automarker.
##
#######

if not SKIP_TESTS:
    fail = True;
    print("{} \'SKIP_TESTS\' is incorrectly set to False.\033[0m".format(issue))
    print("{} You must set the SKIP_TESTS constant to True in the cell above.\033[0m".format(error))
else:
    print('{} \'SKIP_TESTS\' is set to true.\033[0m'.format(success))

#######
##
## Report File Check.
##
## Test that checks there is a report pdf file found in the same folder as the notebook. This is required by the coursework specification.
##
#######

p1 = pathlib.Path('./report.pdf')
p2 = pathlib.Path('./Report.pdf')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("{} Report PDF not found.\033[0m".format(issue))
    print("{} You must include a separate file called report.pdf in your submission.\033[0m".format(error))
else:
    print('{} Report PDF found.\033[0m'.format(success))

#######
##
## File Name check.
##
## Test to ensure file has the correct name. This is important for the marking system to correctly process the submission.
##
#######
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("{} The notebook name is incorrect.\033[0m".format(issue))
    print("{} This notebook file must be named sudoku.ipynb\033[0m".format(error))
else:
    print('{} The notebook name is correct.\033[0m'.format(success))

#######
##
## Create classifier function check.
##
## Test that checks the create_classifier function exists. The function should train the classifier and return it so that it can be evaluated by the marking system.
##
#######

if "sudoku_solver" not in dir():
    fail = True;
    print("{} The sudoku_solver function has not been defined.\033[0m".format(issue))
    print("{} Your code must include a sudoku_solver function as described in the coursework specification.\033[0m".format(error))
    print("{} If you believe you have, \'restart & run-all\' to clear this error.\033[0m".format(error))
else:
    print('{} The sudoku_solver function has been defined.\033[0m'.format(success))



try:
    _sudoku = np.load("data/very_easy_puzzle.npy")[0]
    _solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(_sudoku), _solution):
        print("{} Your sudoku_solver function does not correctly solve the first sudoku.\033[0m".format(issue))
        print()
        print("{} Your assignment is unlikely to get any marks from the autograder. While we will\033[0m".format(error))
        print("{} try to check it manually to assign some partial credit, we encourage you to ask\033[0m".format(error))
        print("{} for help on the forum or directly to a tutor.\033[0m".format(error))
        print()
        print("{} Please use the report file to explain your code anyway.\033[0m".format(error))
    else:
        print("{} Your sudoku_solver function correctly solves the first sudoku.\033[0m".format(success))
        if "__SCORES" in dir():
#             print("{} Test set summary - Not Found.\033[0m".format(issue))
#             print("{} Test set summary could not be found. This is automatically generated when the \033[0m".format(error))
#             print("{} above test cell is run. If you would like to see the summary please run the above cell.\033[0m".format(error))
#             print("{} You do not need this for submission, it is only for your convenience.\033[0m".format(error))
#         else:
            correct = 0
            total = 0
            for key, value in __SCORES.items():
                correct += value['correct']
                total += value['total']
                
            print("{} Test set summary - {}/{} Correct.\033[0m".format(issue, correct, total))
            if total != correct:
                
                for key, value in __SCORES.items():
                    if value['correct'] == value['total']:
                        print("{} {}/{} of {} sudokus correct.\033[0m".format(indent_success, value['correct'], value['total'], key))
                    else:
                        print("{} {}/{} of {} sudokus correct.\033[0m".format(error, value['correct'], value['total'], key))
            
except Exception as e:
    fail = True
    print("{} Error running test set.\033[0m".format(issue))
    print("{} Your code produced the following error. This error will result in a zero from the automarker, please fix.\033[0m".format(error))
    print(e)

    

#######
##
## Final Summary
##
## Prints the final results of the submission tests.
##
#######

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("\033[1m\n\n")
    print("╔═══════════════════════════════════════════════════════════════╗")
    print("║                        Congratulations!                       ║")
    print("║                                                               ║")
    print("║            Your work meets all the required criteria          ║")
    print("║                   and is ready for submission.                ║")
    print("╚═══════════════════════════════════════════════════════════════╝")
    print("\033[0m")
    

In [ ]:
# This is a TEST CELL. Do not delete or change.